In [1]:
from utils.misc import *
from train.train import Train,Test
from datasets.data_utils import load_dataset
from model.Creat_model import creat_model
from utils.preprocess import *
from utils.utils import *

2025-03-31 17:41:31.769024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-31 17:41:31.936189: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-31 17:41:32.851106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:
2025-03-31 17:41:32.851189: W tensorflow/compiler/xla/stream_executor/p

In [2]:
from torch.backends import cudnn
def fix_seed(seed):
    #seed = 2023
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'   

In [3]:
import networkx
import torch
import scipy.sparse as sp
import numpy as np
import os
import random
from munkres import Munkres
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from sklearn.decomposition import PCA
from torch.utils.data import Dataset
import argparse
import yaml
from torch_geometric.utils.convert import to_networkx
import pynvml

def build_args():
    
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--dataset', type=str, default="acm")
    parser.add_argument('--seed', type=int, default=20)
    parser.add_argument('--cuda', type=bool, default=True)
    parser.add_argument('--n_input', type=int, default=None)
    parser.add_argument('--n_z', type=int, default=None)
    parser.add_argument('--freedom_degree', type=float, default=1.0)
    parser.add_argument('--epoch', type=int, default=None)
    parser.add_argument('--shuffle', type=bool, default=True)
    parser.add_argument('--sigma', type=float, default=None)
    parser.add_argument('--loss_n', type=float, default=None)
    parser.add_argument('--loss_w', type=float, default=None)
    parser.add_argument('--loss_s', type=float, default=None)
    parser.add_argument('--loss_a', type=float, default=None)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--acc', type=float, default=-1)
    parser.add_argument('--f1', type=float, default=-1)
    args = parser.parse_args([])
    return args

In [4]:
def pca(adata, use_reps=None, n_comps=10):
    
    """Dimension reduction with PCA algorithm"""
    
    from sklearn.decomposition import PCA
    from scipy.sparse.csc import csc_matrix
    from scipy.sparse.csr import csr_matrix
    pca = PCA(n_components=n_comps)
    if use_reps is not None:
       feat_pca = pca.fit_transform(adata.obsm[use_reps])
    else: 
       if isinstance(adata.X, csc_matrix) or isinstance(adata.X, csr_matrix):
          feat_pca = pca.fit_transform(adata.X.toarray()) 
       else:   
          feat_pca = pca.fit_transform(adata.X)
    
    return feat_pca

def clr_normalize_each_cell(adata, inplace=True):
    
    """Normalize count vector for each cell, i.e. for each row of .X"""

    import numpy as np
    import scipy

    def seurat_clr(x):
        # TODO: support sparseness
        s = np.sum(np.log1p(x[x > 0]))
        exp = np.exp(s / len(x))
        return np.log1p(x / exp)

    if not inplace:
        adata = adata.copy()
    
    # apply to dense or sparse matrix, along axis. returns dense matrix
    adata.X = np.apply_along_axis(
        seurat_clr, 1, (adata.X.A if scipy.sparse.issparse(adata.X) else np.array(adata.X))
    )
    return adata     

In [5]:
args = build_args()
args = load_configs(args, "config/configs.yml")
# set_random_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load configs


In [6]:
import scanpy as sc
import pandas as pd
# read data
file_fold = '/home/hfzhang/data/空间转录组/Mouse_Spleen/' #please replace 'file_fold' with the download path

adata_omics1 = sc.read_h5ad(file_fold + 'adata_RNA.h5ad')
adata_omics2 = sc.read_h5ad(file_fold + 'adata_Pro.h5ad')

adata_omics1.var_names_make_unique()
adata_omics2.var_names_make_unique()
labels=pd.read_csv('/home/hfzhang/data/空间转录组/Mouse_Spleen/annotation_spleen1.csv',header=0,index_col=0)
adata_omics1.obs['final_annot'] = labels.reindex(adata_omics1.obs.index)  
adata_omics2.obs['final_annot'] = labels.reindex(adata_omics2.obs.index) 
adata_omics1 = adata_omics1[adata_omics1.obs['final_annot'].notna()]  
adata_omics2 = adata_omics2[adata_omics2.obs['final_annot'].notna()] 

/home/hfzhang/software/anaconda3/envs/GraphST/lib/python3.8/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [7]:

# RNA
sc.pp.filter_genes(adata_omics1, min_cells=10)
sc.pp.highly_variable_genes(adata_omics1, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_total(adata_omics1, target_sum=1e4)
sc.pp.log1p(adata_omics1)
sc.pp.scale(adata_omics1)

adata_omics1 =  adata_omics1[:, adata_omics1.var['highly_variable']]
adata_omics1.obsm['feat'] = pca(adata_omics1, n_comps=1000)
# Protein
adata_omics2 = clr_normalize_each_cell(adata_omics2)
sc.pp.scale(adata_omics2)
adata_omics2.obsm['feat'] = pca(adata_omics2, n_comps=adata_omics2.n_vars-1)

/home/hfzhang/software/anaconda3/envs/GraphST/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/tmp/ipykernel_54772/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace is deprecated.
  from scipy.sparse.csc import csc_matrix
/tmp/ipykernel_54772/529664883.py:7: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/home/hfzhang/software/anaconda3/envs/GraphST/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/tmp/ipykernel_54772/529664883.py:6: DeprecationWarning: Please use `csc_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csc` namespace i

In [8]:
def create_adj(adata):
    cell_position_omics1 = adata.obsm['spatial']
    adj_omics1 = construct_graph_by_coordinate(cell_position_omics1, n_neighbors=3)
    adata.uns['adj_spatial'] = adj_omics1
    adj_spatial_omics1 = adata.uns['adj_spatial']
    adj_spatial_omics1 = transform_adjacent_matrix(adj_spatial_omics1)
    adj_spatial_omics1 = adj_spatial_omics1.toarray()
    adj_spatial_omics1 = adj_spatial_omics1 + adj_spatial_omics1.T
    adj_spatial_omics1 = np.where(adj_spatial_omics1>1, 1, adj_spatial_omics1)
    adj = preprocess_graph(adj_spatial_omics1)
    return adj

In [9]:
label=adata_omics2.obs['final_annot'].values
classes, label = np.unique(label, return_inverse=True)
classes = classes.tolist()

In [10]:
args.n_input=adata_omics2.n_vars-1
args.n_input1=adata_omics2.n_vars-1
args.n_clusters=len(classes)
device='cuda:1'
args.n_clusters
random_seek=2020
args.Type='Spots'
# args.name='Human_tonsil2'
if args.Type == 'Spots':
    args.epoch=200
    args.loss_n=0.01
elif args.Type == '10x':
    args.loss_n=0.001
    args.epoch=300
elif args.Type == 'Stereo-CITE-seq':
    args.loss_n=0.001
    args.epoch=800
elif args.Type == 'humanplacenta':
    args.loss_n=0.01
    args.epoch=300
args.n_clusters=len(classes)

In [11]:
args.random_seed=random_seek
fix_seed(random_seek)

In [12]:
args.loss_a=0.1
args.loss_n

0.01

In [13]:
# args.epoch=500
# args.ln=0.001
# args.lr=0.0001

In [20]:
args.n_clusters1=len(set(label))
args.n_clusters2=len(set(label))
adj_train=create_adj(adata_omics1)
# adj_test=create_adj(adata_omics1_test)
adj_train = adj_train.to(device)
# adj_test = adj_test.to(device)
features_omics1 = torch.FloatTensor(adata_omics1.obsm['feat'].copy()).to(device)
features_omics2 = torch.FloatTensor(adata_omics2.obsm['feat'].copy()).to(device)
# features_omics1_test = torch.FloatTensor(adata_omics1_test.obsm['feat'].copy()).to(device)
# features_omics2_test = torch.FloatTensor(adata_omics2_test.obsm['feat'].copy()).to(device)

model = creat_model('spamgcn', args).to(device)
# model=preTrain(200, model, features_omics1_train,features_omics2_train, adj_train, label_train, device, args)
model=Train(250, model, features_omics1,features_omics2, adj_train, label, device, args)
nmi, ari, ami, homogeneity, completeness, v_measure=Test(model,adata_omics1,features_omics1,features_omics2,adj_train,label,device,args,'kmeans')
# adata_omics1.write_h5ad('/home/hfzhang/workplace/空间转录组/spaMGCN-main/MGCN-main/save/spaMGCN_spleen_'+str(random_seek)+'.h5ad')


  9 loss: 3.5251760482788086
Epoch_  9 , nmi 0.2054 , ari 0.1016 , ami 0.2036 , homogeneity 0.2483 , completeness 0.1751 , v_measure 0.2054
 19 loss: 3.516737699508667
Epoch_ 19 , nmi 0.2220 , ari 0.1228 , ami 0.2202 , homogeneity 0.2668 , completeness 0.1901 , v_measure 0.2220
 29 loss: 3.5814974308013916
Epoch_ 29 , nmi 0.2202 , ari 0.1190 , ami 0.2184 , homogeneity 0.2654 , completeness 0.1882 , v_measure 0.2202
 39 loss: 3.569789409637451
Epoch_ 39 , nmi 0.2175 , ari 0.1180 , ami 0.2157 , homogeneity 0.2628 , completeness 0.1855 , v_measure 0.2175
 49 loss: 3.5557568073272705
Epoch_ 49 , nmi 0.2205 , ari 0.1100 , ami 0.2187 , homogeneity 0.2653 , completeness 0.1887 , v_measure 0.2205
 59 loss: 3.5387115478515625
Epoch_ 59 , nmi 0.2196 , ari 0.1125 , ami 0.2178 , homogeneity 0.2649 , completeness 0.1875 , v_measure 0.2196
 69 loss: 3.5179829597473145
Epoch_ 69 , nmi 0.2189 , ari 0.1127 , ami 0.2171 , homogeneity 0.2638 , completeness 0.1871 , v_measure 0.2189
 79 loss: 3.4927706718

In [15]:
def eva(y_true, y_pred, epoch=0):  
    if torch.is_tensor(y_true):  
        y_true = y_true.cpu().numpy()  

    acc, f1 = cluster_acc(y_true, y_pred)  
    ami = adjusted_mutual_info_score(y_true, y_pred)  
    nmi = nmi_score(y_true, y_pred, average_method='arithmetic')  
    ari = ari_score(y_true, y_pred)  
    homogeneity = homogeneity_score(y_true, y_pred)  
    completeness = completeness_score(y_true, y_pred)  
    v_measure = v_measure_score(y_true, y_pred)  

    print('Epoch_{:3d}'.format(epoch),  
          ': acc {:.4f}'.format(acc),  
          ', nmi {:.4f}'.format(nmi),  
          ', ari {:.4f}'.format(ari),  
          ', ami {:.4f}'.format(ami),  
          ', homogeneity {:.4f}'.format(homogeneity),  
          ', completeness {:.4f}'.format(completeness),  
          ', v_measure {:.4f}'.format(v_measure))  

In [16]:
a=sc.read_h5ad('/home/hfzhang/workplace/空间转录组/spaMGCN-main/MGCN-main/save/spaMGCN_spleen_2024.h5ad')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/home/hfzhang/workplace/空间转录组/spaMGCN-main/MGCN-main/save/spaMGCN_spleen_2024.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
label=a.obs['final_annot'].values
classes, label = np.unique(label, return_inverse=True)
classes = classes.tolist()

In [ ]:
eva(label,a.obs['pred'])

Epoch_  0 : acc 0.4964 , nmi 0.2156 , ari 0.1750 , ami 0.2137 , homogeneity 0.2467 , completeness 0.1915 , v_measure 0.2156


In [ ]:
ACC: 0.6041, NMI: 0.5324, ARI: 0.4677, AMI: 0.5308, Homogeneity: 0.4608, Completeness: 0.6304, V-measure: 0.5324 thymus  500
CC: 0.5806, NMI: 0.5202, ARI: 0.4693, AMI: 0.5186, Homogeneity: 0.4655, Completeness: 0.5895, V-measure: 0.5202
ACC: 0.5810, NMI: 0.5193, ARI: 0.4692, AMI: 0.5178, Homogeneity: 0.4653, Completeness: 0.5876, V-measure: 0.5193
ACC: 0.5752, NMI: 0.5318, ARI: 0.4839, AMI: 0.5303, Homogeneity: 0.4802, Completeness: 0.5958, V-measure: 0.5318
ACC: 0.5942, NMI: 0.5410, ARI: 0.4754, AMI: 0.5395, Homogeneity: 0.4816, Completeness: 0.6171, V-measure: 0.5410

In [ ]:
acc 0.4964 , nmi 0.2156 , ari 0.1750 , ami 0.2137 , homogeneity 0.2467 , completeness 0.1915 , v_measure 0.2156  spleen
ACC: 0.4904, NMI: 0.2115, ARI: 0.1678, AMI: 0.2096, Homogeneity: 0.2439, Completeness: 0.1867, V-measure: 0.2115 
ACC: 0.4919, NMI: 0.2112, ARI: 0.1712, AMI: 0.2093, Homogeneity: 0.2435, Completeness: 0.1865, V-measure: 0.2112
ACC: 0.5021, NMI: 0.2164, ARI: 0.1761, AMI: 0.2145, Homogeneity: 0.2489, Completeness: 0.1914, V-measure: 0.2164
ACC: 0.4779, NMI: 0.2116, ARI: 0.1604, AMI: 0.2097, Homogeneity: 0.2470, Completeness: 0.1850, V-measure: 0.2116

In [ ]:
Epoch_  0 : acc 0.5669 , nmi 0.4263 , ari 0.3254 , ami 0.4223 , homogeneity 0.4207 , completeness 0.4321 , v_measure 0.4263 300  S3
Epoch_  0 : acc 0.5364 , nmi 0.4298 , ari 0.3165 , ami 0.4258 , homogeneity 0.4304 , completeness 0.4293 , v_measure 0.4298
Epoch_  0 : acc 0.5464 , nmi 0.4188 , ari 0.3085 , ami 0.4148 , homogeneity 0.4160 , completeness 0.4218 , v_measure 0.4188
Epoch_  0 : acc 0.5505 , nmi 0.4220 , ari 0.3117 , ami 0.4213 , homogeneity 0.4174 , completeness 0.4267 , v_measure 0.4220
ACC: 0.5508, NMI: 0.4220, ARI: 0.3126, AMI: 0.4180, Homogeneity: 0.4161, Completeness: 0.4280, V-measure: 0.4220

In [ ]:
Epoch_  0 : acc 0.4906 , nmi 0.3773 , ari 0.2738 , f1 0.3511 , homogeneity 0.4014 , completeness 0.3559 , v_measure 0.3773  600 2024 S2
ACC: 0.5130, NMI: 0.3857, ARI: 0.2853, AMI: 0.3814, Homogeneity: 0.3982, Completeness: 0.3741, V-measure: 0.3857
ACC: 0.4543, NMI: 0.3699, ARI: 0.2517, AMI: 0.3657, Homogeneity: 0.3990, Completeness: 0.3447, V-measure: 0.3699
ACC: 0.5088, NMI: 0.3721, ARI: 0.2734, AMI: 0.3677, Homogeneity: 0.3817, Completeness: 0.3630, V-measure: 0.3721
ACC: 0.5091, NMI: 0.3862, ARI: 0.2853, AMI: 0.3818, Homogeneity: 0.3976, Completeness: 0.3753, V-measure: 0.3862

In [ ]:
Epoch_  0 : acc 0.4613 , nmi 0.4064 , ari 0.2745 , ami 0.4024 , homogeneity 0.4295 , completeness 0.3856 , v_measure 0.4064  300  S1
Epoch_  0 : acc 0.5261 , nmi 0.4218 , ari 0.3077 , ami 0.4179 , homogeneity 0.4355 , completeness 0.4089 , v_measure 0.4218
Epoch_  0 : acc 0.4704 , nmi 0.4006 , ari 0.2710 , ami 0.3966 , homogeneity 0.4222 , completeness 0.3811 , v_measure 0.4006
Epoch_  0 : acc 0.4587 , nmi 0.3979 , ari 0.2731 , ami 0.3939 , homogeneity 0.4254 , completeness 0.3737 , v_measure 0.3979
Epoch_  0 : acc 0.5006 , nmi 0.4054 , ari 0.2803 , ami 0.4014 , homogeneity 0.4160 , completeness 0.3953 , v_measure 0.4054

In [ ]:
nmi 0.5610 , ari 0.4343 , ami 0.5586 , homogeneity 0.5714 , completeness 0.5510 , v_measure 0.5610 E15
nmi 0.5456 , ari 0.4030 , ami 0.5431 , homogeneity 0.5562 , completeness 0.5353 , v_measure 0.5456
nmi 0.5470 , ari 0.4156 , ami 0.5445 , homogeneity 0.5587 , completeness 0.5358 , v_measure 0.5470
nmi 0.5302 , ari 0.3915 , ami 0.5276 , homogeneity 0.5404 , completeness 0.5204 , v_measure 0.5302
nmi 0.5583 , ari 0.4161 , ami 0.5558 , homogeneity 0.5675 , completeness 0.5494 , v_measure 0.5583

In [ ]:
print(f"ACC: {acc:.4f}, NMI: {nmi:.4f}, ARI: {ari:.4f}, AMI: {ami:.4f}, Homogeneity: {homogeneity:.4f}, Completeness: {completeness:.4f}, V-measure: {v_measure:.4f}")

ACC: 0.4779, NMI: 0.2116, ARI: 0.1604, AMI: 0.2097, Homogeneity: 0.2470, Completeness: 0.1850, V-measure: 0.2116
